# Three Agent Types Demo - Microsoft Agent Framework (MAF)

This notebook demonstrates the **three types of agents** available in Microsoft Agent Framework and their different chat history management approaches.

## What You'll Learn

1. **Azure AI Agent** (`AzureAIClient`) - Service-managed threads stored server-side
2. **Azure OpenAI Chat Completion** (`AzureOpenAIChatClient`) - Client-managed history with full control
3. **Azure OpenAI Responses** (`AzureOpenAIResponsesClient`) - Flexible history (both service and custom)

Each demo shows:
- How to create and configure each agent type
- Environment variables required for each
- Multi-turn conversations demonstrating memory/context handling
- When to use each agent type based on your requirements

## Prerequisites

- Python 3.9+ with packages: `agent-framework`, `agent-framework-azure-ai`, `azure-identity`
- Azure CLI authentication: Run `az login` before executing cells
- Azure AI Foundry project (for Azure AI Agent)
- Azure OpenAI resource (for Chat and Responses agents)

**Note**: Each agent type uses different environment variable names - see setup cell below for details.

## Setup: Load Environment Variables

In [2]:
import os
from pathlib import Path
from dotenv import load_dotenv

# Load .env file
env_path = Path.cwd() / '.env'
if env_path.exists():
    load_dotenv(env_path)
    print("✅ Loaded .env file")
else:
    print("⚠️  No .env file found - using system environment variables")

# Verify required variables
print("\n=== Azure AI Agent Variables ===")
print(f"AZURE_AI_PROJECT_ENDPOINT: {'✅ Set' if os.getenv('AZURE_AI_PROJECT_ENDPOINT') else '❌ Missing'}")
print(f"AZURE_AI_MODEL_DEPLOYMENT_NAME: {'✅ Set' if os.getenv('AZURE_AI_MODEL_DEPLOYMENT_NAME') else '❌ Missing'}")

print("\n=== Azure OpenAI Chat Completion Variables ===")
print(f"AZURE_OPENAI_ENDPOINT: {'✅ Set' if os.getenv('AZURE_OPENAI_ENDPOINT') else '❌ Missing'}")
print(f"AZURE_OPENAI_CHAT_DEPLOYMENT_NAME: {'✅ Set' if os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME') else '❌ Missing'}")

print("\n=== Azure OpenAI Responses Variables ===")
print(f"AZURE_OPENAI_ENDPOINT: {'✅ Set' if os.getenv('AZURE_OPENAI_ENDPOINT') else '❌ Missing'}")
print(f"AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME: {'✅ Set' if os.getenv('AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME') else '❌ Missing'}")

print("\n⚠️  Note: Each agent type uses DIFFERENT deployment variable names!")
print("See: AQ-CODE/docs/AGENT_ENV_VARS_EXPLAINED.md for details")

✅ Loaded .env file

=== Azure AI Agent Variables ===
AZURE_AI_PROJECT_ENDPOINT: ✅ Set
AZURE_AI_MODEL_DEPLOYMENT_NAME: ✅ Set

=== Azure OpenAI Chat Completion Variables ===
AZURE_OPENAI_ENDPOINT: ✅ Set
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME: ✅ Set

=== Azure OpenAI Responses Variables ===
AZURE_OPENAI_ENDPOINT: ✅ Set
AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME: ✅ Set

⚠️  Note: Each agent type uses DIFFERENT deployment variable names!
See: AQ-CODE/docs/AGENT_ENV_VARS_EXPLAINED.md for details


## 1. Azure AI Agent (`AzureAIClient`)

**Uses**: Azure AI Agents Service (azure-ai-projects SDK backend)  
**Chat History**: Service-managed only (threads stored server-side)  
**Best For**: Production scenarios with hosted infrastructure

**Environment Variables Needed**:
- `AZURE_AI_PROJECT_ENDPOINT`
- `AZURE_AI_MODEL_DEPLOYMENT_NAME`

In [3]:
import asyncio
from agent_framework.azure import AzureAIClient
from azure.identity.aio import AzureCliCredential

async def demo_azure_ai_agent():
    print("\n=== Azure AI Agent Demo ===")
    
    # Check required environment variables
    if not os.getenv('AZURE_AI_PROJECT_ENDPOINT'):
        print("❌ Missing AZURE_AI_PROJECT_ENDPOINT")
        return
    
    # Create credential first
    credential = AzureCliCredential()
    
    async with AzureAIClient(async_credential=credential).create_agent(
        name="AzureAIAgent",
        instructions="You are a helpful assistant. Keep responses concise.",
    ) as agent:
        print("✅ Azure AI Agent created successfully")
        print("📝 Sending query...")
        
        # Note: No thread needed for single-turn interactions
        # For multi-turn conversations, pass thread=AgentThread()
        result = await agent.run("What type of agent are you? Answer in one sentence.")
        print(f"🤖 Response: {result.text}")
        print(f"💾 Chat History: Service-managed (thread ID: {result.thread_id if hasattr(result, 'thread_id') else 'N/A'})")


await demo_azure_ai_agent()


=== Azure AI Agent Demo ===


ServiceInitializationError: Azure credential is required when project_client is not provided.

## 1b. Azure AI Agent - Explicit Thread Management (Advanced Pattern - V2 API)

The previous demo used `agent.run()` which abstracts thread management. This demo shows the **low-level Azure AI Foundry Agents V2 API** using `AIProjectClient` for explicit control over:

- **Threads**: Conversation sessions (max 100,000 messages per thread)
- **Messages**: Individual communications added to threads
- **Runs**: Agent invocations to process thread messages
- **Run Status**: Monitoring execution (queued → in_progress → completed)

**What is the V2 API?**

Azure AI Foundry Agents V2 introduces:
- ✅ **Agent Versioning**: Each agent configuration is saved as an immutable version
- ✅ **`create_version()` method**: Replaces the older `create_agent()` approach
- ✅ **`PromptAgentDefinition`**: Structured agent configuration
- ✅ **Version History**: Track all changes, compare configurations, and rollback
- ✅ **Production Stability**: Immutable versions prevent accidental overwrites

**Why use this pattern?**
- Production systems requiring external integrations
- Custom logging, monitoring, or debugging workflows
- Explicit thread persistence and lifecycle management
- Access to intermediate run states
- Version control for agent configurations

**What you'll see:**
The code below implements the complete 7-step workflow using V2 API: create versioned agent → create thread → add message → create run → poll status → retrieve messages → follow-up interaction → cleanup.

In [ ]:

from azure.ai.projects.aio import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition
from azure.identity.aio import AzureCliCredential

async def demo_explicit_thread_management():
    """
    Demonstrates explicit thread, run, and message management
    using Azure AI Foundry Agents V2 API.
    
    V2 API Features:
    - Agent versioning with create_version()
    - Immutable agent versions for production stability
    - PromptAgentDefinition for agent configuration
    - Version-based agent retrieval and management
    """
    print("\n=== Azure AI Agent - Explicit Thread Management (V2 API) ===")
    
    if not os.getenv('AZURE_AI_PROJECT_ENDPOINT'):
        print("❌ Missing AZURE_AI_PROJECT_ENDPOINT")
        return
    
    async with (
        AzureCliCredential() as credential,
        AIProjectClient(
            endpoint=os.getenv('AZURE_AI_PROJECT_ENDPOINT'),
            credential=credential
        ) as project_client
    ):
        # Step 1: Create an agent using V2 API (create_version)
        print("\n1️⃣ Creating agent (V2 API - versioned agent)...")
        agent_name = "ThreadDemoAgent"
        agent_version = await project_client.agents.create_version(
            agent_name=agent_name,
            definition=PromptAgentDefinition(
                model=os.getenv('AZURE_AI_MODEL_DEPLOYMENT_NAME', 'gpt-4o'),
                instructions="You are a helpful assistant demonstrating thread management."
            ),
            description="Demo agent for explicit thread management (V2 API)"
        )
        print(f"   ✅ Agent created: {agent_version.name} (version {agent_version.version})")
        print(f"   📦 V2 Feature: This version is immutable and can be referenced as '{agent_name}:{agent_version.version}'")
        
        # Get the agent to retrieve its actual agent ID (asst_xxx format)
        agent = await project_client.agents.get(agent_name=agent_name, agent_version=agent_version.version)
        agent_id = agent.agent_id  # This should be in format 'asst_xxx'
        print(f"   🔑 Agent ID: {agent_id}")
        
        # For thread/message/run operations, we need the standalone AgentsClient
        from azure.ai.agents.aio import AgentsClient
        agents_client = AgentsClient(
            endpoint=os.getenv('AZURE_AI_PROJECT_ENDPOINT'),
            credential=credential
        )
        
        # Step 2: Create a thread (conversation session)
        print("\n2️⃣ Creating thread...")
        thread = await agents_client.threads.create()
        print(f"   ✅ Thread created: {thread.id}")
        print(f"   💾 This thread persists on server - can resume later")
        
        # Step 3: Add a message to the thread
        print("\n3️⃣ Adding message to thread...")
        message = await agents_client.messages.create(
            thread_id=thread.id,
            role="user",
            content="What are the key benefits of thread-based conversation management?"
        )
        print(f"   ✅ Message added: {message.id}")
        
        # Step 4: Create and process run (use the actual agent ID)
        print("\n4️⃣ Creating and processing run...")
        run = await agents_client.runs.create_and_process(
            thread_id=thread.id,
            agent_id=agent_id  # Use the actual agent ID (asst_xxx) from V2 API
        )
        print(f"   ✅ Run completed: {run.status}")
        
        if run.status == "failed":
            print(f"   ❌ Run failed: {run.last_error}")
            await project_client.agents.delete(agent_name=agent_name, agent_version=agent_version.version)
            await agents_client.close()
            return
        
        # Step 5: Retrieve and display messages
        print("\n5️⃣ Retrieving messages from thread...")
        messages = await agents_client.messages.list(thread_id=thread.id)
        
        print("\n📝 Conversation:")
        for msg in messages.data:
            role = "🙋 User" if msg.role == "user" else "🤖 Agent"
            if msg.text_messages:
                for text_msg in msg.text_messages:
                    print(f"{role}: {text_msg.text.value}")
        
        # Demonstrate thread persistence - add another message
        print("\n6️⃣ Adding follow-up message (shows thread persistence)...")
        await agents_client.messages.create(
            thread_id=thread.id,
            role="user",
            content="Can you summarize the key point in one sentence?"
        )
        
        # Create another run
        run2 = await agents_client.runs.create_and_process(
            thread_id=thread.id,
            agent_id=agent_id  # Use the actual agent ID
        )
        
        if run2.status == "failed":
            print(f"   ❌ Run failed: {run2.last_error}")
        else:
            # Get updated messages
            messages = await agents_client.messages.list(thread_id=thread.id)
            for msg in messages.data:
                if msg.role == "assistant" and msg.run_id == run2.id:
                    if msg.text_messages:
                        for text_msg in msg.text_messages:
                            print(f"🤖 Follow-up response: {text_msg.text.value}")
                    break
        
        # Cleanup
        print("\n7️⃣ Cleanup...")
        await project_client.agents.delete(agent_name=agent_name, agent_version=agent_version.version)
        await agents_client.close()
        print("   ✅ Agent deleted")
        print(f"   💡 Thread {thread.id} persists server-side until explicitly deleted")
        print(f"   📦 V2 Feature: Version {agent_version.version} is immutable - deletion removes entire agent")

# Run the demo
await demo_explicit_thread_management()


### Demo 1b: Explicit Thread Management Implementation

This cell demonstrates the **low-level Azure AI Foundry Agent API** using `AIProjectClient` directly. Unlike `agent.run()` which abstracts thread management, this shows the complete 8-step workflow:

1. Create agent → 2. Create thread → 3. Add message → 4. Create run → 5. Poll status → 6. Retrieve messages → 7. Follow-up interaction → 8. Cleanup

This pattern gives you full control over the agent lifecycle and is recommended for production scenarios where you need to:
- Integrate with external systems
- Implement custom logging or monitoring
- Manage thread persistence explicitly
- Access intermediate run states for debugging

**Run the cell below to see each step in action:**

## 1b. Azure AI Agent - Explicit Thread Management (Advanced)

The previous demo used `agent.run()` which abstracts thread management. For production scenarios, you may want explicit control over threads, runs, and messages as described in the [Azure AI Foundry documentation](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/concepts/threads-runs-messages).

**Key Concepts:**
- **Thread**: A conversation session storing messages (max 100,000 messages per thread)
- **Messages**: Individual communications (text, files) within a thread
- **Run**: Invocation of the agent to process thread messages and generate responses
- **Run Status**: Monitor run lifecycle (queued → in_progress → completed)

This pattern is useful when you need:
- Fine-grained control over thread lifecycle
- Access to intermediate run states
- Custom message handling or storage
- Integration with external systems tracking conversation state

### Visual Workflow

![Azure AI Agent Run Workflow](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/media/run-thread-model.png?view=foundry-classic)

*Source: [Microsoft Learn - Threads, runs, and messages](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/concepts/threads-runs-messages?view=foundry-classic)*

The diagram above shows the complete lifecycle of an agent interaction using explicit thread management. Our demo below implements all these steps programmatically.

## 2. Azure OpenAI Chat Completion (`AzureOpenAIChatClient`)

**Uses**: Azure OpenAI Chat Completion API  
**Chat History**: Custom/client-managed only (you control storage)  
**Best For**: Custom storage requirements, full control over conversation state

**Environment Variables Needed**:
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME` ⚠️ Note: **CHAT**_DEPLOYMENT
- `AZURE_OPENAI_API_VERSION` (optional)

In [ ]:
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

async def demo_azure_openai_chat():
    print("\n=== Azure OpenAI Chat Completion Agent Demo ===")
    
    # Check required environment variables
    if not os.getenv('AZURE_OPENAI_ENDPOINT'):
        print("❌ Missing AZURE_OPENAI_ENDPOINT")
        return
    if not os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME'):
        print("❌ Missing AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
        print("💡 This is different from AZURE_OPENAI_DEPLOYMENT_NAME!")
        return
    
    agent = AzureOpenAIChatClient(credential=AzureCliCredential()).create_agent(
        name="ChatCompletionAgent",
        instructions="You are a helpful assistant. Keep responses concise.",
    )
    
    print("✅ Azure OpenAI Chat Completion Agent created successfully")
    print("📝 Sending query...")
    
    # Note: No thread needed for single-turn interactions
    # For multi-turn conversations, pass thread=AgentThread()
    result = await agent.run("What type of agent are you? Answer in one sentence.")
    print(f"🤖 Response: {result.text}")
    print(f"💾 Chat History: Client-managed (you must implement storage)")


# Run the demoawait demo_azure_openai_chat()

## 3. Azure OpenAI Responses (`AzureOpenAIResponsesClient`)

**Uses**: Azure OpenAI Responses API  
**Chat History**: Both service-managed AND custom (most flexible!)  
**Best For**: Scenarios requiring both convenience and customization

**Environment Variables Needed**:
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME` ⚠️ Note: **RESPONSES**_DEPLOYMENT
- `AZURE_OPENAI_API_VERSION` (optional)

In [ ]:
from agent_framework.azure import AzureOpenAIResponsesClient
from azure.identity import AzureCliCredential

async def demo_azure_openai_responses():
    print("\n=== Azure OpenAI Responses Agent Demo ===")
    
    # Check required environment variables
    if not os.getenv('AZURE_OPENAI_ENDPOINT'):
        print("❌ Missing AZURE_OPENAI_ENDPOINT")
        return
    
    # Check if endpoint is Azure OpenAI (not Azure AI Foundry)
    endpoint = os.getenv('AZURE_OPENAI_ENDPOINT', '')
    if not endpoint.endswith('.openai.azure.com/') and not endpoint.endswith('.openai.azure.com'):
        print("⚠️  Azure OpenAI Responses API is NOT available on Azure AI Foundry endpoints")
        print(f"   Your endpoint: {endpoint}")
        print("   Required: https://your-resource.openai.azure.com/")
        print("\n💡 The Responses API only works with native Azure OpenAI Service endpoints.")
        print("💡 Since you're using Azure AI Foundry, this demo will be skipped.")
        print("💡 Use AzureAIClient (Demo #1) or AzureOpenAIChatClient (Demo #2) instead.")
        return
    
    if not os.getenv('AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME'):
        print("❌ Missing AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME")
        print("💡 This is different from AZURE_OPENAI_DEPLOYMENT_NAME!")
        print("💡 Set: AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME=your-deployment-name")
        return
    
    try:
        agent = AzureOpenAIResponsesClient(credential=AzureCliCredential()).create_agent(
            name="ResponsesAgent",
            instructions="You are a helpful assistant. Keep responses concise.",
        )
        
        print("✅ Azure OpenAI Responses Agent created successfully")
        print("📝 Sending query...")
        
        # Note: No thread needed for single-turn interactions
        # For multi-turn conversations, pass thread=AgentThread()
        result = await agent.run("What type of agent are you? Answer in one sentence.")
        print(f"🤖 Response: {result.text}")
        print(f"💾 Chat History: Service-managed OR custom (your choice!)")
    except Exception as e:
        print(f"❌ Error: {e}")
        print("\n💡 This typically means the Responses API is not available on your endpoint.")


# Run the demoawait demo_azure_openai_responses()

## Comparison: Chat History Management

Let's demonstrate multi-turn conversations with each agent type:

In [ ]:
async def demo_multi_turn_conversations():
    print("\n=== Multi-Turn Conversation Comparison ===")
    
    # Import async credential and AgentThread for maintaining conversation state
    from azure.identity.aio import AzureCliCredential as AsyncAzureCliCredential
    from agent_framework import AgentThread
    
    # Azure AI Agent - Service-managed history
    if os.getenv('AZURE_AI_PROJECT_ENDPOINT'):
        print("\n1️⃣ Azure AI Agent (Service-managed history):")
        
        # Create credential first
        credential = AsyncAzureCliCredential()
        
        async with AzureAIClient(async_credential=credential).create_agent(
            name="MultiTurnAgent",
            instructions="Remember the conversation context.",
        ) as agent:
            # Create a thread to maintain conversation state
            thread = AgentThread()
            
            # First turn
            result1 = await agent.run("My favorite color is blue.", thread=thread)
            print(f"   User: My favorite color is blue.")
            print(f"   Agent: {result1.text}")
            
            # Second turn (agent should remember because we're using the same thread)
            result2 = await agent.run("What's my favorite color?", thread=thread)
            print(f"   User: What's my favorite color?")
            print(f"   Agent: {result2.text}")
            print(f"   ✅ Service remembers context via AgentThread object")
    
    # Azure OpenAI Chat - Client-managed history
    if os.getenv('AZURE_OPENAI_ENDPOINT') and os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME'):
        print("\n2️⃣ Azure OpenAI Chat (Client-managed history):")
        agent = AzureOpenAIChatClient(credential=AzureCliCredential()).create_agent(
            instructions="Remember the conversation context.",
        )
        
        # Create a thread to maintain conversation state
        thread = AgentThread()
        
        result1 = await agent.run("My favorite color is blue.", thread=thread)
        print(f"   User: My favorite color is blue.")
        print(f"   Agent: {result1.text}")
        
        result2 = await agent.run("What's my favorite color?", thread=thread)
        print(f"   User: What's my favorite color?")
        print(f"   Agent: {result2.text}")
        print(f"   ✅ AgentThread stores history client-side")
        
    # Azure OpenAI Responses - Flexible history
    if os.getenv('AZURE_OPENAI_ENDPOINT') and os.getenv('AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME'):
        endpoint = os.getenv('AZURE_OPENAI_ENDPOINT', '')
        if endpoint.endswith('.openai.azure.com/') or endpoint.endswith('.openai.azure.com'):
            print("\n3️⃣ Azure OpenAI Responses (Flexible history):")
            agent = AzureOpenAIResponsesClient(credential=AzureCliCredential()).create_agent(
                instructions="Remember the conversation context.",
            )
            
            # Create a thread to maintain conversation state
            thread = AgentThread()
            
            result1 = await agent.run("My favorite color is blue.", thread=thread)
            print(f"   User: My favorite color is blue.")
            print(f"   Agent: {result1.text}")
            
            result2 = await agent.run("What's my favorite color?", thread=thread)
            print(f"   User: What's my favorite color?")
            print(f"   Agent: {result2.text}")
            print(f"   ✅ Can use service OR custom history via AgentThread")
        else:
            print("\n3️⃣ Azure OpenAI Responses (Skipped - requires .openai.azure.com endpoint)")

# Run the demo
await demo_multi_turn_conversations()

## Summary: When to Use Each Agent Type

### ✅ Use Azure AI Agent when:
- You want service-managed conversation threads
- Building production apps with Azure AI Foundry/Projects
- Need hosted agent infrastructure
- Want automatic thread management

### ✅ Use Azure OpenAI Chat Completion when:
- You need full control over chat history storage
- Implementing custom persistence (database, Redis, etc.)
- Want to manage conversation state manually
- Building custom chat applications

### ✅ Use Azure OpenAI Responses when:
- You want the flexibility of both options
- Need structured response generation
- Want to choose between service or custom history
- Building applications that may evolve requirements

---

## Complete .env File Template

⚠️ **CRITICAL**: Each agent type uses **different deployment variable names**!

Create a `.env` file in your project root with these variables:

```bash
# ========================================
# Azure AI Agent (AzureAIClient)
# ========================================
AZURE_AI_PROJECT_ENDPOINT="https://your-ai-services.services.ai.azure.com/api/projects/your-project"
AZURE_AI_MODEL_DEPLOYMENT_NAME="gpt-4o"

# ========================================
# Azure OpenAI Chat Completion (AzureOpenAIChatClient)
# ========================================
AZURE_OPENAI_ENDPOINT="https://your-resource.openai.azure.com/"
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME="gpt-4o"        # Note: CHAT_DEPLOYMENT
AZURE_OPENAI_API_VERSION="2024-10-21"

# ========================================
# Azure OpenAI Responses (AzureOpenAIResponsesClient)
# ========================================
# Uses same AZURE_OPENAI_ENDPOINT as above
AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME="gpt-4o"   # Note: RESPONSES_DEPLOYMENT

# Authentication: Run 'az login' in terminal
```

**Important**: 
- Don't commit your `.env` file to git! Add it to `.gitignore`.
- See `AQ-CODE/docs/AGENT_ENV_VARS_EXPLAINED.md` for detailed explanation of variable naming.

## 🎓 Key Learnings from This Demo

### 1. Conversation State Management
**Critical Discovery**: Without passing an `AgentThread` object, each `agent.run()` call is completely independent - the agent has NO memory of previous interactions.

```python
# ❌ Wrong - Each call is a new conversation
result1 = await agent.run("My favorite color is blue.")
result2 = await agent.run("What's my favorite color?")  # Agent won't remember!

# ✅ Correct - Use AgentThread to maintain conversation state
from agent_framework import AgentThread
thread = AgentThread()
result1 = await agent.run("My favorite color is blue.", thread=thread)
result2 = await agent.run("What's my favorite color?", thread=thread)  # Agent remembers!
```

### 2. Environment Variable Naming Confusion
Each agent type uses **different** deployment variable names. This is intentional to allow using different models for different agent types:

| Agent Type | Deployment Variable Name |
|------------|-------------------------|
| Azure AI Agent | `AZURE_AI_MODEL_DEPLOYMENT_NAME` |
| Azure OpenAI Chat | `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME` |
| Azure OpenAI Responses | `AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME` |

**Why?** This allows flexibility:
```bash
# You can use different models for different purposes
AZURE_AI_MODEL_DEPLOYMENT_NAME="gpt-4o"              # Production agent
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME="gpt-4o-mini"      # Cost-effective chat
AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME="gpt-4o"      # Advanced responses
```

### 3. Azure OpenAI Responses API Endpoint Requirements
The Responses API has strict endpoint requirements:
- ✅ **Works with**: `https://your-resource.openai.azure.com/` (Native Azure OpenAI)
- ❌ **Does NOT work with**: `https://your-resource.services.ai.azure.com/` (Azure AI Foundry)

**Why?** The Responses API is a preview feature only available on native Azure OpenAI Service endpoints. The SDK checks for `.openai.azure.com` suffix before allowing the API call.

### 4. Credential Initialization Pattern for Azure AI Agent
Azure AI Agent requires creating the credential before passing it to the client:

```python
# ❌ Wrong - Using async context manager for credential
async with AzureCliCredential() as credential:
    async with AzureAIClient(async_credential=credential).create_agent(...) as agent:

# ✅ Correct - Create credential first, then use it
credential = AzureCliCredential()
async with AzureAIClient(async_credential=credential).create_agent(...) as agent:
```

**Why?** The `AzureAIClient` needs an initialized credential at construction time, not one that's being set up in a context manager.

### 5. Chat History Storage Differences

| Agent Type | History Storage | When to Use |
|------------|----------------|-------------|
| **Azure AI Agent** | Service-managed (server-side) | Production apps, want Azure to handle persistence |
| **Azure OpenAI Chat** | Client-managed (you store it) | Custom storage (DB, Redis), full control |
| **Azure OpenAI Responses** | Both options available | Most flexible, can choose per use case |

**Important**: Even though storage differs, all three use `AgentThread()` locally to maintain conversation state during execution.

### 6. Authentication Pattern
All demos use Azure CLI authentication:
1. Run `az login` in terminal
2. Use `AzureCliCredential()` in code
3. Azure handles authentication automatically

This is the recommended approach for development and testing. Production apps should use Managed Identity or Service Principal authentication.

---

💡 **See Also**: [AGENT_ENV_VARS_EXPLAINED.md](../docs/AGENT_ENV_VARS_EXPLAINED.md) for detailed explanation of environment variable naming patterns.

## References

- [Microsoft Learn - Agent Types](https://learn.microsoft.com/en-us/agent-framework/user-guide/agents/agent-types/?pivots=programming-language-python)
- [MAF GitHub - Azure AI Samples](https://github.com/microsoft/agent-framework/tree/main/python/samples/getting_started/agents/azure_ai)
- [MAF GitHub - Azure OpenAI Samples](https://github.com/microsoft/agent-framework/tree/main/python/samples/getting_started/agents/azure_openai)